In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import pickle
import time
import dill
import numpy as np
#http://www.newyorksocialdiary.com/party-pictures?page=1
#url = "http://www.newyorksocialdiary.com/party-pictures"
#response = requests.get(url, params={"limit": 1000, "offset": 0})
#response = requests.get(url,params={'page':3})
#print response.url
#print response.text[:200] + "..."

In [177]:

def scrap1(soup):
    parent_div = soup.select('div[class=view-content]')[0] #Find (at most) *one*
    a=parent_div.select('span[class=field-content]')
    #print a
    N=len(a)
    event=[a[i].text for i in range(N) if (i % 2)==0]
    date=[a[i].text for i in range(N) if (i % 2)==1]
    link=[a[i].select('a')[0]['href'] for i in range (N) if (i%2)==0]
    df=pd.DataFrame({'event':event,'date':date,'link':link})
    return (df)


df=pd.DataFrame(columns=['event','date','link'])
for page in range (27):
    response = requests.get("http://www.newyorksocialdiary.com/party-pictures",params={'page':page})
    soup = BeautifulSoup(response.text,"lxml")
    ans=scrap1(soup)
    df=pd.concat([df,ans])

#Updating index

df.date=pd.DatetimeIndex(df.date)
df.index=df.date

before=datetime.strptime("30/11/14", "%d/%m/%y")
df=df.ix[before:]
df

    


,date,event,link
date,,,
2014-11-24,2014-11-24,Gala Guests,/party-pictures/2014/gala-guests
2014-11-20,2014-11-20,Equal Justice,/party-pictures/2014/equal-justice
2014-11-18,2014-11-18,Celebrating the Treasures,/party-pictures/2014/celebrating-the-treasures
2014-11-17,2014-11-17,Associates and Friends,/party-pictures/2014/associates-and-friends
2014-11-13,2014-11-13,Michael's 25th,/party-pictures/2014/michaels-25th
2014-11-12,2014-11-12,New York Lifelines,/party-pictures/2014/new-york-lifelines
2014-11-10,2014-11-10,Legends and Leaders,/party-pictures/2014/legends-and-leaders
2014-11-06,2014-11-06,Fall Fun,/party-pictures/2014/fall-fun
2014-11-05,2014-11-05,Park Avenue Armory's 2014 Gala Masquerade,/party-pictures/2014/park-avenue-armorys-2014-...


In [ ]:
sel_divphotocaption = 'div.photocaption'
sel_divphotocaptionshrp = 'div#photocaption'    
sel_trphotocaption = 'tr.photocaption'
sel_trphotocaptionshrp = 'tr#photocaption'
sel_tdphotocaption = 'td.photocaption'
sel_tdphotocaptionshrp = 'td#photocaption'    
sel_spanphotocaption = 'span.photocaption'
sel_tddivfont = 'td > div > font'

In [178]:
links=["http://www.newyorksocialdiary.com" + df.link[i] for i in range(len(df))]

In [193]:
def caption_list3(url):
    x=0
    while x<10: # Trying 10 max for each URL before exiting
        response = requests.get(url)
        if response.status_code==200:
            break
        else:
            x+=1
    if x==10:
        return []
    
    soup = BeautifulSoup(response.text)
    captions_list=[]
    sel_divphotocaption = 'div.photocaption'
    sel_divphotocaptionshrp = 'div#photocaption'    
    sel_trphotocaption = 'tr.photocaption'
    sel_trphotocaptionshrp = 'tr#photocaption'
    sel_tdphotocaption = 'td.photocaption'
    sel_tdphotocaptionshrp = 'td#photocaption'    
    sel_spanphotocaption = 'span.photocaption'
    sel_tddivfont = 'td > div > font'
    
    
    
    captions = soup.select('div[class=photocaption]')  #First caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    
    captions = soup.select('font[size="1"]')  #First caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    #for fontfrag in soup.select('font[size="1"]'):
     #       if fontfrag.string is None:
    #            continue
     #       captions_list.append(fontfrag.string.strip())
    captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"}) #Second caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    captions = soup.findAll("td", {"class": "photocaption","scope":"row"}) #Third caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    captions = soup.select('td[class=photocaption]') #Fourth caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #aptions_list.extend(captions_list)
    

    if len(captions_list)>0:
        #print url
        return (captions_list,len(captions_list))
    else:
        print url
        return []

In [200]:
def caption_list4(url):
    x=0
    while x<10: # Trying 10 max for each URL before exiting
        response = requests.get(url)
        if response.status_code==200:
            break
        else:
            x+=1
    if x==10:
        return []
    
    soup = BeautifulSoup(response.text)
    captions_list=[]
    sel_divphotocaption = 'div.photocaption'
    sel_divphotocaptionshrp = 'div#photocaption'    
    sel_trphotocaption = 'tr.photocaption'
    sel_trphotocaptionshrp = 'tr#photocaption'
    sel_tdphotocaption = 'td.photocaption'
    sel_tdphotocaptionshrp = 'td#photocaption'    
    sel_spanphotocaption = 'span.photocaption'
    sel_tddivfont = 'td > div > font'
    
    
    
    captions = soup.select(sel_divphotocaption)  #First caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    
    captions = soup.select(sel_divphotocaptionshrp)  #First caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    #for fontfrag in soup.select('font[size="1"]'):
     #       if fontfrag.string is None:
    #            continue
     #       captions_list.append(fontfrag.string.strip())
    captions = soup.findAll(sel_trphotocaption) #Second caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    captions = soup.findAll(sel_trphotocaptionshrp) #Third caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    captions = soup.select(sel_tdphotocaption) #Fourth caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #aptions_list.extend(captions_list)
    captions = soup.select(sel_tdphotocaptionshrp) #Fourth caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    
    captions = soup.select(sel_spanphotocaption) #Fourth caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    
    
    captions = soup.select(sel_tddivfont) #Fourth caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    
    
    if len(captions_list)>0:
        #print url
        return (captions_list)
    else:
        print url
        return []

In [211]:
def caption_list (url): #Retrieves url, returns a list of photo captions
    x=0
    while x<10: # Trying 10 max for each URL before exiting
        response = requests.get(url)
        if response.status_code==200:
            break
        else:
            x+=1
    if x==10:
        return []
    
    soup = BeautifulSoup(response.text)
    captions_list=[]
    
    captions = soup.select('div[class=photocaption]')  #First caption type
    if len(captions)>0:
        captions_list=[captions[i].text for i in range (len(captions))]
    else:
        captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"}) #Second caption type
        if len(captions)>0:
            captions_list=[captions[i].text for i in range (len(captions))]
        else:
            captions = soup.findAll("td", {"class": "photocaption","scope":"row"}) #Third caption type
            if len(captions)>0:
                captions_list=[captions[i].text for i in range (len(captions))]
            else: 
                captions = soup.select('td[class=photocaption]') #Fourth caption type
                if len(captions)>0:
                    captions_list=[captions[i].text for i in range (len(captions))]
    
    if len(captions_list)>0:
        return (captions_list,len(captions_list))
    else:
        print url
        return []
    

d=[] #the caption containers
counter=0
for url in links:
    time.sleep(0.1)
    if len(caption_list4(url))>0:
        counter+=1
        d.extend(caption_list4(url))
    
    if counter%50==0: #Printing and saving files
        print(counter)
        outfile=open( str(counter+1), "wb" )
        pickle.dump(d, outfile)
        outfile.close()
    #a=all_captions + caption_list(url)
outfile=open( "raw_captions4", "wb" )
pickle.dump(d, outfile)
outfile.close()  
#url="http://www.newyorksocialdiary.com/party-pictures/2014/public-celebrations"
##a=caption_list(url)
#a

http://www.newyorksocialdiary.com/nysd/partypictures
50
100
150
200
250
http://www.newyorksocialdiary.com/party-pictures/2012/simple-pleasures
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150


In [212]:
len(d)

131898

In [196]:
d[4]


[u'Fujiko Nakaya\'s "Veil" envelops the  Glass House.',
 u' Harpist Mary Lattimore',
 u' Charles Renfro, Tony Vidler, James Welling, and Daniel Gortler',
 u' Margaret Russell, Henry Urbach, and Margareth Henriquez',
 u' Carlos Souza and Charlene de Ganay',
 u' Bonnie Morrison and Douglas Friedman',
 u' Procession over the eyebrow bridge on way to Painting Gallery',
 u' Martha Stewart and Charles Renfro',
 u' Margaret Russell, Martha Stewart, Susan Magrino Dunning, Margareth Henriquez, Reed Krakoff, Emma O\u2019Neill, John Yunis, and John Calcagno',
 u' Lee Mindel, Margaret Russell, Tomas Maier, Martha Stewart, Susan Magrino Dunning',
 u' Bill Katz, Bonnie Morrison, Douglas Friedman, Henry Urbach, Laurie Beckelman, James Sanders, Tony Vidler, and Charles Renfro',
 u' Monica Rich Kosann and Reed Krakoff',
 u' Hendel Teicher',
 u' Henry Urbach and Charles Renfro',
 u' Lori Tritsch and William Lauder',
 u' Margaret Russell in the Sculpture Gallery',
 u' Martha Stewart, Laura Pla, Tomas Mai

In [188]:
a1=[]
for i in range(len(d)):
    if i%2==1:
        a1.append(d[i])
np.max(a1)

445

In [194]:
a1[1]

83

In [185]:
url="http://www.newyorksocialdiary.com/party-pictures/2014/opening-numbers"

print url
response = requests.get(url)
soup=BeautifulSoup(response.text)
#captions = soup.findAll("td", {"class": "photocaption","scope":"row"})
#captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"})
#captions = soup.select('div[class=photocaption]')
#captions = soup.select('td[class=photocaption]')
#captions=soup.findAll("div", {"class":["class1", "class2"]})
#caption_list=[captions[i].text for i in range (len(captions))]
d=caption_list3(url)
d

http://www.newyorksocialdiary.com/party-pictures/2014/opening-numbers
http://www.newyorksocialdiary.com/party-pictures/2014/opening-numbers


[u'Susan Burke, and Eleanor Sullivan']

In [76]:
outfile=open( "raw_captions2", "r" )
d=pickle.load(outfile)
outfile.close()  


In [77]:
a=d[:]

In [78]:
a

[u'The scene at   IDEAL School & Academy\u2019s  10th\xa0Annual Gala.',
 u' Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, and Ron Iervolino ',
 u' Chuck Grodin ',
 u' Diana Rosario, Ali Sussman, Sarah Boll, Jen Zaleski, Alysse Brennan, and Lindsay Macbeth ',
 u' Kelly  and Tom Murro ',
 u' Udo Spreitzenbarth ',
 u' Ron Iervolino, Trish Iervolino, Russ Middleton, and Lisa Middleton ',
 u' Barbara Loughlin, Dr. Gerald Loughlin, and Debbie Gelston ',
 u' Julianne Michelle ',
 u' Heather Robinson, Kiwan Nichols, Jimmy Nichols, Melanie Carbone, and Nancy Brown ',
 u' Bill Mack and Les Lieberman ',
 u' Dr. David Lyden and Patricia Sorenson ',
 u' Jimmy Cayne, Vince Tese, and Pat Cayne ',
 u' Stuart Oran, Les Lieberman, and Hilary Oran ',
 u' Vince Tese and Chuck Grodin ',
 u' Dwight Gooden and Les Lieberman ',
 u' Dr. Amy Cunningham-Bussel, Ray Mirra, and Dr. Tyler Janovitz ',
 u' Dan Shedrick and Samara Heafitz ',
 u' Cass and Jason Adelman ',
 u' Bart Scott and Mark Lapl

In [79]:
#Removing empty and very long captions
a = filter(None, a)
a = [a[i] for i in range (len(a))  if len(a[i])<=250]
len(a)

117192

In [80]:
b=a[:]
#b

In [81]:
#Converting connection words to ",

for i, caption in enumerate(b):
    b[i]=caption.replace("Gillian and Sylvester Miniter","Gillian Miniter and Sylvester Miniter")

for i, caption in enumerate(b):
    b[i]=caption.replace("Sylvester and Gillian Miniter","Gillian Miniter and Sylvester Miniter")

for i, caption in enumerate(b):
    b[i]=caption.replace("Peter and Jamee Gregory","Peter Gregory and Jamee Gregory")

for i, caption in enumerate(b):
    b[i]=caption.replace("Jamee and Peter Gregory","Peter Gregory and Jamee Gregory")
    
for i, caption in enumerate(b):
    b[i]=caption.replace("Donald and Barbara Tober","Donald Tober and Barbara Tober")
    
for i, caption in enumerate(b):
    b[i]=caption.replace("Barbara and Donald Tober","Donald Tober and Barbara Tober")

for i, caption in enumerate(b):
    b[i]=caption.replace("Jean and Martin Shafiroff","Jean Shafiroff and Martin Shafiroff")

for i, caption in enumerate(b):
    b[i]=caption.replace("Martin and Jean Shafiroff","Jean Shafiroff and Martin Shafiroff")

for i, caption in enumerate(b):
    b[i]=caption.replace("Tatiana and Campion Platt","Tatiana Platt and Campion Platt")

for i, caption in enumerate(b):
    b[i]=caption.replace("Campion and Tatiana Platt","Tatiana Platt and Campion Platt")
    
    
for i, caption in enumerate(b):
    b[i]=caption.replace("Valentin and Yaz Hernandez","Valentin Hernandez and Yaz Hernandez")
    
for i, caption in enumerate(b):
    b[i]=caption.replace("Yaz and Valentine Hernandez","Valentin Hernandez and Yaz Hernandez")


for i, caption in enumerate(b):
    b[i]=caption.replace("Somers and Jonathan Farkas","Somers Farkas and Jonathan Farkas")

for i, caption in enumerate(b):
    b[i]=caption.replace("Jonathan and Somer Farkas","Somers Farkas and Jonathan Farkas")



for i, caption in enumerate(b):
    b[i]=caption.replace('and',',')
for i, caption in enumerate(b):
    b[i]=caption.replace(";",",")
for i, caption in enumerate(b):
    b[i]=caption.replace("&",",")
for i, caption in enumerate(b):
    b[i]=caption.replace(" with ",",")
for i, caption in enumerate(b):
    b[i]=caption.replace('"',",")
for i, caption in enumerate(b):
    b[i]=caption.replace('/',",")
for i, caption in enumerate(b):
    b[i]=caption.replace('Co-chairs','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Mayor','')
for i, caption in enumerate(b):
    b[i]=caption.replace('M.D.','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Dr.','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Ambassador','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Princess','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Mr ','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Mrs ','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Mr. ','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Mrs. ','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Gala Chair','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Gala Vice Chair','')
for i, caption in enumerate(b):
    b[i]=caption.replace('Honoree','')

    
    


In [82]:
b

[u'The scene at   IDEAL School , Academy\u2019s  10th\xa0Annual Gala.',
 u' Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, , Ron Iervolino ',
 u' Chuck Grodin ',
 u' Diana Rosario, Ali Sussman, Sarah Boll, Jen Zaleski, Alysse Brennan, , Lindsay Macbeth ',
 u' Kelly  , Tom Murro ',
 u' Udo Spreitzenbarth ',
 u' Ron Iervolino, Trish Iervolino, Russ Middleton, , Lisa Middleton ',
 u' Barbara Loughlin,  Gerald Loughlin, , Debbie Gelston ',
 u' Julianne Michelle ',
 u' Heather Robinson, Kiwan Nichols, Jimmy Nichols, Melanie Carbone, , Nancy Brown ',
 u' Bill Mack , Les Lieberman ',
 u'  David Lyden , Patricia Sorenson ',
 u' Jimmy Cayne, Vince Tese, , Pat Cayne ',
 u' Stuart Oran, Les Lieberman, , Hilary Oran ',
 u' Vince Tese , Chuck Grodin ',
 u' Dwight Gooden , Les Lieberman ',
 u'  Amy Cunningham-Bussel, Ray Mirra, ,  Tyler Janovitz ',
 u' Dan Shedrick , Samara Heafitz ',
 u' Cass , Jason Adelman ',
 u' Bart Scott , Mark Lapl,er ',
 u' Mitch Rubin, Audra Zuckerman, Mic

In [59]:
#Replacing white space with ","
#for i, caption in enumerate(b):
#    b[i]=caption.replace(" ",",")
len(b)

117192

In [83]:
#Splitting into list of lists
for i, caption in enumerate(b):
    b[i]=caption.split(",")
#c=reduce(lambda x, y: x+y, b)
#c
b

[[u'The scene at   IDEAL School ', u' Academy\u2019s  10th\xa0Annual Gala.'],
 [u' Les Lieberman',
  u' Barri Lieberman',
  u' Isabel Kallman',
  u' Trish Iervolino',
  u' ',
  u' Ron Iervolino '],
 [u' Chuck Grodin '],
 [u' Diana Rosario',
  u' Ali Sussman',
  u' Sarah Boll',
  u' Jen Zaleski',
  u' Alysse Brennan',
  u' ',
  u' Lindsay Macbeth '],
 [u' Kelly  ', u' Tom Murro '],
 [u' Udo Spreitzenbarth '],
 [u' Ron Iervolino',
  u' Trish Iervolino',
  u' Russ Middleton',
  u' ',
  u' Lisa Middleton '],
 [u' Barbara Loughlin', u'  Gerald Loughlin', u' ', u' Debbie Gelston '],
 [u' Julianne Michelle '],
 [u' Heather Robinson',
  u' Kiwan Nichols',
  u' Jimmy Nichols',
  u' Melanie Carbone',
  u' ',
  u' Nancy Brown '],
 [u' Bill Mack ', u' Les Lieberman '],
 [u'  David Lyden ', u' Patricia Sorenson '],
 [u' Jimmy Cayne', u' Vince Tese', u' ', u' Pat Cayne '],
 [u' Stuart Oran', u' Les Lieberman', u' ', u' Hilary Oran '],
 [u' Vince Tese ', u' Chuck Grodin '],
 [u' Dwight Gooden ', u' L

In [51]:
for cap in b[:200]:
    for i in range(len(cap)):
        for name in cap:
            if (len(name.split())==1) & (i<len(cap)-2):
                print name + cap[i+1].split()[1]

er Lapl
erHoll


IndexError: list index out of range

In [ ]:
ans=[]
for i in range(len(b)):
    for j in range (len(b[i])):
        name=b[i][j]
        qq=name.split()
        if (len(qq)==4) & ('and' in qq):
            ans.extend(b[i][j])
            print (b[i][j])
            #b[i][j]=name[0] + name[4] + name[2] + name[3] + name[4]

ans
            #ans           

 Kelly  and Tom Murro 
 Cass and Jason Adelman 
 Dana and Jamie Creel 
 and Alexia Hamm Ryan 
 Michael  and Jenny Price 
 Bob  and Suzanne Cochran 
 Patsy and Jeff Tarr 
 Fred and Stephanie Shuman 
 Gillian and Sylvester Miniter 
 Michael  and Lise Evans 
 Fe  and Alessandro Fendi 
 Nancy and Dan Paduano 
 Bill and Anne Harrison 
 Mitchell  and Mary Silver 
Yvonne and Geoffrey Canada
Denise and Bernard Tyson
Debby and Jack Greenberg
 and Debra Martin Chase
Tasha Smith and friend
 Peter and Andrea Klein
 and Franklin Hill Perrell 
 Lisa and Gilbert Ott
 Carol and Jim Large
 Thomas and Devon Carroll
 Christine and John Gachot
 Ed and Pat Travaglianti
Carol and Ed Clarke 
 Thomas and Diahn McGrath
Robert and Jon Cooper 
 John and Bonnie Williamson
 Rusty and Peggy McCormack
 Doug and Chris Fox
 Brendan and Kathleen FitzGerald 
 Jim and Eileen Cahill
MaryAnn and Peter Hurd 
 Peri and Martin Wenz
 Bob and Debbie Hussey
 Lynda and Jacques Busquet
 and Ann Marie Mirabile 
 Rebecca and Ben Mal

In [130]:
len (ans)

100

In [84]:
e=b[:]
e

[[u'The scene at   IDEAL School ', u' Academy\u2019s  10th\xa0Annual Gala.'],
 [u' Les Lieberman',
  u' Barri Lieberman',
  u' Isabel Kallman',
  u' Trish Iervolino',
  u' ',
  u' Ron Iervolino '],
 [u' Chuck Grodin '],
 [u' Diana Rosario',
  u' Ali Sussman',
  u' Sarah Boll',
  u' Jen Zaleski',
  u' Alysse Brennan',
  u' ',
  u' Lindsay Macbeth '],
 [u' Kelly  ', u' Tom Murro '],
 [u' Udo Spreitzenbarth '],
 [u' Ron Iervolino',
  u' Trish Iervolino',
  u' Russ Middleton',
  u' ',
  u' Lisa Middleton '],
 [u' Barbara Loughlin', u'  Gerald Loughlin', u' ', u' Debbie Gelston '],
 [u' Julianne Michelle '],
 [u' Heather Robinson',
  u' Kiwan Nichols',
  u' Jimmy Nichols',
  u' Melanie Carbone',
  u' ',
  u' Nancy Brown '],
 [u' Bill Mack ', u' Les Lieberman '],
 [u'  David Lyden ', u' Patricia Sorenson '],
 [u' Jimmy Cayne', u' Vince Tese', u' ', u' Pat Cayne '],
 [u' Stuart Oran', u' Les Lieberman', u' ', u' Hilary Oran '],
 [u' Vince Tese ', u' Chuck Grodin '],
 [u' Dwight Gooden ', u' L

In [87]:
#removing private names only and some other one-word trash
for i, nest in enumerate(e):
    for j, person in enumerate(nest):
        #print person, len(person.split())
        if len(person.split())<=1:
            #print i,j
            e[i][j]="xxxxxx"


In [88]:
#removing white spaces on both sides
for i, nest in enumerate(e):
    for j, person in enumerate(nest):
        #print person, len(person.split())
        e[i][j]=person.strip()
e

[[u'The scene at   IDEAL School', u'Academy\u2019s  10th\xa0Annual Gala.'],
 [u'Les Lieberman',
  u'Barri Lieberman',
  u'Isabel Kallman',
  u'Trish Iervolino',
  'xxxxxx',
  u'Ron Iervolino'],
 [u'Chuck Grodin'],
 [u'Diana Rosario',
  u'Ali Sussman',
  u'Sarah Boll',
  u'Jen Zaleski',
  u'Alysse Brennan',
  'xxxxxx',
  u'Lindsay Macbeth'],
 ['xxxxxx', u'Tom Murro'],
 [u'Udo Spreitzenbarth'],
 [u'Ron Iervolino',
  u'Trish Iervolino',
  u'Russ Middleton',
  'xxxxxx',
  u'Lisa Middleton'],
 [u'Barbara Loughlin', u'Gerald Loughlin', 'xxxxxx', u'Debbie Gelston'],
 [u'Julianne Michelle'],
 [u'Heather Robinson',
  u'Kiwan Nichols',
  u'Jimmy Nichols',
  u'Melanie Carbone',
  'xxxxxx',
  u'Nancy Brown'],
 [u'Bill Mack', u'Les Lieberman'],
 [u'David Lyden', u'Patricia Sorenson'],
 [u'Jimmy Cayne', u'Vince Tese', 'xxxxxx', u'Pat Cayne'],
 [u'Stuart Oran', u'Les Lieberman', 'xxxxxx', u'Hilary Oran'],
 [u'Vince Tese', u'Chuck Grodin'],
 [u'Dwight Gooden', u'Les Lieberman'],
 [u'Amy Cunningham-Bus

In [89]:
#removing caption with only one person
#r=e[:]

for i, caption in enumerate(e):
    if len(caption)<=1:
        e[i]=['xxxxxx']
        #del(e[i])


In [90]:
#removing xxxxxxx externally
g=e[:]
while ['xxxxxx'] in g:
    g.remove(['xxxxxx'])


In [91]:
#removing xxxxxx from within the lists
for i, captions in enumerate(g):
    while 'xxxxxx' in captions:
        g[i].remove('xxxxxx')


In [92]:
g

[[u'The scene at   IDEAL School', u'Academy\u2019s  10th\xa0Annual Gala.'],
 [u'Les Lieberman',
  u'Barri Lieberman',
  u'Isabel Kallman',
  u'Trish Iervolino',
  u'Ron Iervolino'],
 [u'Diana Rosario',
  u'Ali Sussman',
  u'Sarah Boll',
  u'Jen Zaleski',
  u'Alysse Brennan',
  u'Lindsay Macbeth'],
 [u'Tom Murro'],
 [u'Ron Iervolino', u'Trish Iervolino', u'Russ Middleton', u'Lisa Middleton'],
 [u'Barbara Loughlin', u'Gerald Loughlin', u'Debbie Gelston'],
 [u'Heather Robinson',
  u'Kiwan Nichols',
  u'Jimmy Nichols',
  u'Melanie Carbone',
  u'Nancy Brown'],
 [u'Bill Mack', u'Les Lieberman'],
 [u'David Lyden', u'Patricia Sorenson'],
 [u'Jimmy Cayne', u'Vince Tese', u'Pat Cayne'],
 [u'Stuart Oran', u'Les Lieberman', u'Hilary Oran'],
 [u'Vince Tese', u'Chuck Grodin'],
 [u'Dwight Gooden', u'Les Lieberman'],
 [u'Amy Cunningham-Bussel', u'Ray Mirra', u'Tyler Janovitz'],
 [u'Dan Shedrick', u'Samara Heafitz'],
 [u'Jason Adelman'],
 [u'Bart Scott', u'Mark Lapl'],
 [u'Mitch Rubin',
  u'Audra Zucke

In [84]:
#all_names=reduce(lambda x, y: x+y, g)
#unique_names=list(set(all_names))
#len(unique_names)

KeyboardInterrupt: 

In [65]:
from collections import Counter
Counter(all_names).most_common(10)

NameError: name 'all_names' is not defined

In [93]:
from itertools import combinations
import networkx as nx
G=nx.Graph()
G
for cap in g:
    for comb in combinations(cap,2):
        if G.has_edge(comb[0],comb[1]):
            G.edge[comb[0]][comb[1]]['weight'] += 1
        else:
            G.add_edge(comb[0], comb[1], weight=1)

In [94]:
GG=G.edges(data='weight')
s=sorted(GG, key=lambda gg: gg[2], reverse=True)[:100]
type(s)

ans=[]
for i in s:
    ans.append(((i[0],i[1]),i[2]))
ans

[((u'Sylvester Miniter', u'Gillian Miniter'), 116),
 ((u'Bonnie Comley', u'Stewart Lane'), 86),
 ((u'Peter Gregory', u'Jamee Gregory'), 77),
 ((u'Andrew Saffir', u'Daniel Benedict'), 71),
 ((u'Geoffrey Bradfield', u'Roric Tobin'), 69),
 ((u'Campion Platt', u'Tatiana Platt'), 61),
 ((u'Donald Tober', u'Barbara Tober'), 59),
 ((u'Jay Diamond', u'ra Lebenthal'), 55),
 ((u'Jean Shafiroff', u'Martin Shafiroff'), 50),
 ((u'Sessa von Richthofen', u'Richard Johnson'), 46),
 ((u'Valentin Hern', u'Yaz Hern'), 45),
 ((u'Deborah Norville', u'Karl Wellner'), 43),
 ((u'Ruth L', u'e Shuman'), 41),
 ((u'Elizabeth Stribling', u'Guy Robinson'), 41),
 ((u'Hilary Geary Ross', u'Wilbur Ross'), 39),
 ((u'a Kellogg', u'Kirk Henckels'), 34),
 ((u'Frederick Anderson', u'Douglas Hannant'), 34),
 ((u'Somers Farkas', u'Jonathan Farkas'), 34),
 ((u'Michael Cominotto', u'Dennis Basso'), 31),
 ((u'Janna Bullock', u'R. Couri Hay'), 31),
 ((u'Leonel Piraino', u'Nina Griscom'), 30),
 ((u'Arlene Dahl', u'Marc Rosen'), 2

In [118]:
qq=G.degree()
ans=[(key,qq[key]) for key in qq.keys()]
s=sorted(ans, key=lambda gg: gg[1], reverse=True)[:100]
s
#list(G.degree().values())
#list(G.degree().keys())

[(u'Jean Shafiroff', 400),
 (u'Mark Gilbertson', 328),
 (u'Gillian Miniter', 242),
 (u'Geoffrey Bradfield', 211),
 (u'Michael Bloomberg', 203),
 (u'Alexandra Lebenthal', 201),
 (u'Andrew Saffir', 191),
 (u'Mario Buatta', 190),
 (u'Alina Cho', 187),
 (u'Patrick McMullan', 187),
 (u'Somers Farkas', 185),
 (u'Sharon Bush', 179),
 (u'Yaz Hernandez', 179),
 (u'Lucia Hwong Gordon', 170),
 (u'Debbie Bancroft', 166),
 (u'Kamie Lightburn', 163),
 (u'Muffie Potter Aston', 156),
 (u'Allison Aston', 153),
 (u'Bettina Zilkha', 151),
 (u'Eleanora Kennedy', 149),
 (u'Jamee Gregory', 143),
 (u'Christopher Hyland', 139),
 (u'Martha Stewart', 137),
 (u'Lydia Fenet', 137),
 (u'Kipton Cronkite', 134),
 (u'Barbara Tober', 134),
 (u'Michele Herbert', 132),
 (u'Karen LeFrak', 130),
 (u'Amy Fine Collins', 130),
 (u'Deborah Norville', 129),
 (u'Evelyn Lauder', 127),
 (u'Leonard Lauder', 125),
 (u'Audrey Gruss', 124),
 (u'Karen Klopp', 123),
 (u'Nicole Miller', 123),
 (u'Dennis Basso', 123),
 (u'Russell Simmons

In [41]:
s[1]

(u'Andrew Saffir', u'Daniel Benedict', 71)

In [29]:
from operator import itemgetter
pp2=sorted(G.edges(data='weight').iteritems(), key=itemgetter(1), reverse=True)
pp2[:100]

AttributeError: 'list' object has no attribute 'iteritems'

In [47]:
a={}
ttt=0
for name in unique_names:
        count=0     
        ttt+=1
        if (ttt % 100)==0:
            print ttt
        for cap in g:
            if name in cap:
                count+=1
        a[name]=count        


100
200
300
400
500
600


KeyboardInterrupt: 

In [63]:
#creating caption dictionary
d1={}
for key,value in enumerate(g):
    d1[key]=value


In [64]:
len(d1)

90033

In [105]:
pp={}
ppf={}
count=0

for name in unique_names:
    pp[name]=[]
    ppf[name]=[]
    #print name
    for i in range(len (d1)):
        if name in d1[i]:
            friends=d1[i]
            #print d[i]
            ppf[name].extend(friends)
    pp[name]=set(ppf[name])








MemoryError: 

In [67]:
ppf

{u'Actress Peri Gilpin Vincent': [u'Actress Peri Gilpin Vincent',
  u'Christian Vincent'],
 u'Adam Chanler-Berat': [u'Celia Keenan-Bolger',
  u'Adam Chanler-Berat',
  u'Celia Keenan-Bolger',
  u'Adam Chanler-Berat'],
 u'Adrian Gordon': [u'Jeffrey Morrison', u'Adrian Gordon'],
 u'Alexandra Baker': [u'Emily Connor', u'Alexandra Baker', u'Norma Quintero'],
 u'Amira Thoron': [u'Will Harcourt-Smith',
  u'Amira Thoron',
  u'Elise Enders',
  u'Tom Enders'],
 u'Ana Leibowitz': [u'Bridget McDermott',
  u'Robert Del Greco',
  u'Ana Leibowitz'],
 u'Angela Power': [u'John Beinecke',
  u'Angela Power',
  u'Robert Power.',
  u'John Beinecke',
  u'Angela Power',
  u'Robert Power.'],
 u'Anne Mulden': [u'Amanda Kahn', u'Anne Mulden', u'Alexandra Sagalas'],
 u'Arlene Skierawski': [u'KC Williams', u'Arlene Skierawski'],
 u'Ashley Meinders': [u'Ashley Meinders', u'Inda Grosseld', u'Brianna Iovino'],
 u'Autumn Hurlbert': [u'Lindsey Ridgway', u'Bailey Hanks', u'Autumn Hurlbert'],
 u'Averell Satloff': [u'Ave

In [104]:
import networkx as nx
G=nx.Graph()
G
for name in ppf.keys():
    for friend in ppf[name]:
        
        if G.has_edge(friend,name):
            G.edge[friend][name]['weight'] += 1
        else:
            G.add_edge(friend, name, weight=1 )
            #G.add_edge(name,friend)
G.edges(data='weight')

[(u'Christie Fraser', u'Christie Fraser', 1),
 (u'Christie Fraser', u'Liz Celeste', 1),
 (u'Dennis Serras', u'Dennis Serras', 1),
 (u'Dennis Serras', u'Marlo Bakst', 1),
 (u'Dennis Serras', u'Mikhail Baryshnikov', 2),
 (u'Dennis Serras', u'Tara Nasta', 1),
 (u'Ryan Swardstrom', u'Ryan Swardstrom', 1),
 (u'Ryan Swardstrom', u'Tom Dolby', 1),
 (u'Ryan Swardstrom', u'Peter Spears', 1),
 (u'Joyce Azria-Nassir', u'Joyce Azria-Nassir', 1),
 (u'Joyce Azria-Nassir', u'Marine Azria', 1),
 (u'Leslie Kirschenbaum', u'Leslie Kirschenbaum', 2),
 (u'Leslie Kirschenbaum', u'Elise Zealand', 2),
 (u'Arlene Skierawski', u'Arlene Skierawski', 1),
 (u'Arlene Skierawski', u'KC Williams', 1),
 (u'Jennifer Lantzas', u'Jennifer Lantzas', 1),
 (u'Jennifer Lantzas', u'Will Ryman', 1),
 (u'Jennifer Lantzas', u'Ashley Ludwig', 1),
 (u'Jennifer Lantzas', u'Jonathan Kuhn', 1),
 (u'Pearson Chodak', u'Pearson Chodak', 1),
 (u'Pearson Chodak', u'Jonathan Sheffer', 1),
 (u'Jackie Terrebonne', u'Jolene Travis', 1),
 (u'

In [ ]:
G.has_edge()

In [265]:
#for i in len(pp):
pp1={}
for key in pp.keys():
    pp1[key]=(len(pp[key])-1)


In [266]:
from operator import itemgetter
pp2=sorted(pp1.iteritems(), key=itemgetter(1), reverse=True)
pp2[:100]

[(u'Jean Shafiroff', 400),
 (u'Mark Gilbertson', 328),
 (u'Gillian Miniter', 242),
 (u'Geoffrey Bradfield', 211),
 (u'Michael Bloomberg', 203),
 (u'Alexandra Lebenthal', 201),
 (u'Andrew Saffir', 191),
 (u'Mario Buatta', 190),
 (u'Alina Cho', 187),
 (u'Patrick McMullan', 187),
 (u'Somers Farkas', 185),
 (u'Sharon Bush', 179),
 (u'Yaz Hernandez', 179),
 (u'Lucia Hwong Gordon', 170),
 (u'Debbie Bancroft', 166),
 (u'Kamie Lightburn', 163),
 (u'Muffie Potter Aston', 154),
 (u'Allison Aston', 153),
 (u'Bettina Zilkha', 151),
 (u'Eleanora Kennedy', 149),
 (u'Jamee Gregory', 143),
 (u'Christopher Hyland', 139),
 (u'Martha Stewart', 137),
 (u'Lydia Fenet', 137),
 (u'Kipton Cronkite', 134),
 (u'Barbara Tober', 134),
 (u'Michele Herbert', 132),
 (u'Karen LeFrak', 130),
 (u'Amy Fine Collins', 130),
 (u'Deborah Norville', 129),
 (u'Evelyn Lauder', 127),
 (u'Leonard Lauder', 125),
 (u'Audrey Gruss', 124),
 (u'Karen Klopp', 123),
 (u'Nicole Miller', 123),
 (u'Dennis Basso', 123),
 (u'Russell Simmons

In [227]:
pd.Series([i[1] for i in pp2]).describe()

count    58737.000000
mean         6.968589
std         11.717051
min          0.000000
25%          2.000000
50%          4.000000
75%          8.000000
max        584.000000
dtype: float64

In [228]:
def bff(friends):
    d2={}
    unique_friends=list(set(friends))
    for name in unique_friends:
          d2[name]=friends.count(name)
    return d2


In [305]:
bff_dict={}
for name in ppf.keys():
    bff_dict[name]=bff(ppf[name])
bff_dict

{u'Tom Smarr': {u'Barbara Paul Robinson': 1,
  u'Claudia Bonn': 1,
  u'Louis Bauer': 1,
  u'Marechal Brown': 1,
  u'Mark Fisher': 1,
  u'Tom Smarr': 1},
 u'Christie Fraser': {u'Christie Fraser': 1, u'Liz Celeste': 1},
 u'Dennis Serras': {u'Dennis Serras': 1,
  u'Marlo Bakst': 1,
  u'Mikhail Baryshnikov': 1,
  u'Tara Nasta': 1},
 u'Honorees: Richard C. Hunt': {u'Andy Jagoda': 1,
  u'Honorees: Richard C. Hunt': 1,
  u'Kevin W. Chason': 1,
  u'Susan C. Waltman': 1},
 u'Samuel Barnett': {u'Jano Herbosch': 1,
  u'Joseph Timms': 1,
  u'Samuel Barnett': 1},
 u'Jennifer Lantzas': {u'Ashley Ludwig': 1,
  u'Jennifer Lantzas': 1,
  u'Jonathan Kuhn': 1,
  u'Will Ryman': 1},
 u'Pearson Chodak': {u'Jonathan Sheffer': 1, u'Pearson Chodak': 1},
 u'Sarah Julig': {u'Sarah Julig': 1, u'Tom Finkelpearl': 1},
 u'Jackie Terrebonne': {u'Jackie Terrebonne': 1,
  u'Jolene Travis': 1,
  u'Lee Cavanaugh': 1},
 u'Jaime E. Jim\xe9nez of Baccarat': {u'Chair Clare McKeon': 1,
  u'Chair Sloan Overstrom': 1,
  u'Jaime

In [306]:
ans3={}
for person in bff_dict.keys():
    for friend in bff_dict[person].keys():
            if person!=friend:
                ans3[person+','+friend]=bff_dict[person][friend] * 2
ans3

{u'Bob Wankel,Joe Benincasa': 2,
 u'Betty Gorab,Esra Munnell': 2,
 u'Astrid Misrahi,Gala Co-Chair Judy Zankel': 2,
 u'Dylan Walsh,Bryan Batt': 2,
 u'Noah Chesnin,Barbara Pryor': 2,
 u'Anita Contini,Gala Co-Chair Frances Resheske': 2,
 u'Tim Daly,Michael Frankfurt': 2,
 u'Danielle Smotrich,Yigal Azrouel': 2,
 u'Laurie Malman,Nancy Moonves': 2,
 u'Balasz Nyari,Francine Nyari': 2,
 u'Gennaro Vitally,Theresa Beasley': 2,
 u'Lydia Fenet,Mary Giuliani': 6,
 u'Ricardo Vollbrechthausen,Adrian Deseglise': 2,
 u'Dr. Feder,Daniella Nik': 2,
 u'Kim Bates,Victoria Holloway': 2,
 u'Nikki M. James,Joel Silverman': 2,
 u'Dr. Bonnie M. Davis,Peter W. May': 2,
 u'Maureen Robertson,Rachel Leonard': 2,
 u'Gladys George,Dr. Stuart Orsher': 6,
 u'Leigh Bishop Taub,Katie Holmes.': 2,
 u'Cold Pina Coladas,Francois Payard': 2,
 u'Elena Phillips,Sarah Ramsey': 2,
 u'John W. Munson,Eiseley Tauginas': 2,
 u'Judy Libe,Emmy Lou Kelly': 2,
 u'Charlotte Ford,Dr. Steven J. Corwin': 2,
 u"Natalie Pfanstiehl is a well-k

In [302]:
from operator import itemgetter
ans4=sorted(ans3.iteritems(), key=itemgetter(1), reverse=True)
ans4[:100]
list(ans4)
x=ans4
for i in range (200):
    if (i % 2 ==0):
        x=ans4[i]
        print str(tuple([tuple(x[0].split(',')),x[1]])) + ","

#print tuple([tuple(x[0][0].split(',')),x[0][1]])
#ans4[0][1]
#tuple(x) for x in 
pd.Series([i[1] for i in ans4[:100]]).describe()

((u'Bonnie Comley', u'Stewart Lane'), 108),
((u'Andrew Saffir', u'Daniel Benedict'), 104),
((u'Roric Tobin', u'Geoffrey Bradfield'), 102),
((u'Richard Johnson', u'Sessa von Richthofen'), 54),
((u'Karl Wellner', u'Deborah Norville'), 52),
((u'Wilbur Ross', u'Hilary Geary Ross'), 50),
((u'Guy Robinson', u'Elizabeth Stribling'), 50),
((u'Elizabeth Stribling', u'Guy Robinson'), 50),
((u'Douglas Hannant', u'Frederick Anderson'), 48),
((u'Olivia Palermo', u'Johannes Huebl'), 46),
((u'Marc Rosen', u'Arlene Dahl'), 44),
((u'Anne Hearst McInerney', u'Jay McInerney'), 42),
((u'Leonel Piraino', u'Nina Griscom'), 38),
((u'Gillian Miniter', u'Sylvester Miniter'), 36),
((u'John Catsimatidis', u'Margo Catsimatidis'), 34),
((u'Brian Stewart', u'Stephanie Krieger'), 34),
((u'Stephanie Krieger', u'Brian Stewart'), 34),
((u'David Lauren', u'Lauren Bush'), 32),
((u'Jean Shafiroff', u'Sharon Bush'), 32),
((u'ra of Greece', u'Princess Alex'), 30),
((u'Michael Cominotto', u'Dennis Basso'), 30),
((u'Tony Ingr

count    100.000000
mean      34.580000
std       19.282682
min       20.000000
25%       24.000000
50%       26.000000
75%       36.500000
max      108.000000
dtype: float64

(u'Bonnie Comley,Stewart Lane', 108)

In [119]:
prank=nx.pagerank(G)

In [120]:
prank1=sorted(prank.iteritems(), key=itemgetter(1), reverse=True)
prank1[:100]
#pd.Series([i[1] for i in prank1[:100]]).describe()

[(u'Jean Shafiroff', 0.000634100059060083),
 (u'Mark Gilbertson', 0.000499950694634983),
 (u'Gillian Miniter', 0.0004269715875654436),
 (u'Geoffrey Bradfield', 0.00037526006309356936),
 (u'Andrew Saffir', 0.00033993962033088105),
 (u'Alexandra Lebenthal', 0.00033096616924577845),
 (u'Sharon Bush', 0.0003097674003915945),
 (u'Michael Bloomberg', 0.00029204223542145713),
 (u'Mario Buatta', 0.00029170590628565927),
 (u'Somers Farkas', 0.00028958400369728685),
 (u'Yaz Hernandez', 0.00028125545452624645),
 (u'Alina Cho', 0.0002794282256507939),
 (u'Debbie Bancroft', 0.0002787600345467584),
 (u'Patrick McMullan', 0.0002711942911992435),
 (u'Lucia Hwong Gordon', 0.0002511435437617871),
 (u'a friend', 0.0002490321829594809),
 (u'Bonnie Comley', 0.000248101601394993),
 (u'Kamie Lightburn', 0.00024805555226275566),
 (u'Christopher Hyland', 0.00023656951358769858),
 (u'Photographs by PatrickMcMullan.com', 0.00023352177300375434),
 (u'Eleanora Kennedy', 0.0002318055451904091),
 (u'his wife', 0.000

In [89]:
import networkx as nx
G=nx.Graph()
G
for name in ppf.keys():
    for friend in ppf[name]:
        G.add_edge(name,friend)


355

In [ ]:
del 